#Modern AI Pro: Types of memory
We will use different types of memory to store our conversations.

In [ ]:
from google.colab import userdata
!pip install git+https://gitlab.com/gauthammsam/mitrallm.git --quiet
from mitrallm import MitraLLM
llm = MitraLLM(
    token     = userdata.get("MITRA_TOKEN"),
    accessapi = userdata.get("MITRA_ENDPOINT")
)
print(llm)

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.4/252.4 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 17.3 MB/s eta 0:00:00
MitraLLM
Params: {'URL': 'https://mitrallm.mitrarobot.com'}


## Basic: Conversation Buffer Memory

In [ ]:
!pip install -q langchain
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(return_messages=True)
memory.load_memory_variables({})



{'history': []}

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

In [ ]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

from operator import itemgetter

chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
    )
    | prompt
    | llm
)

In [ ]:
from langchain.callbacks.tracers import ConsoleCallbackHandler

input = {"input": "I want to understand more about AI"}
response = chain.invoke({"input":input }, config={'callbacks': [ConsoleCallbackHandler()]})
response

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "input": {
    "input": "I want to understand more about AI"
  }
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableAssign<history>] Entering Chain run with input:
{
  "input": {
    "input": "I want to understand more about AI"
  }
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableAssign<history> > 3:chain:RunnableParallel<history>] Entering Chain run with input:
{
  "input": {
    "input": "I want to understand more about AI"
  }
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableAssign<history> > 3:chain:RunnableParallel<history> > 4:chain:RunnableSequence] Entering Chain run with input:
{
  "input": {
    "input": "I want to understand more about AI"
  }
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableAssign<history> > 3:chain:RunnableParallel<history> > 4:chain:RunnableSequence > 5:chain:load_memory_variables] Entering Chain run with input:
{
  "input": {
    "input"

"System: Sure, I'd be happy to help! AI stands for artificial intelligence"

In [ ]:
memory.save_context(input, {"output": response})
memory.load_memory_variables({})

{'history': [HumanMessage(content='I want to understand more about AI'),
  AIMessage(content="System: Sure, I'd be happy to help! AI stands for artificial intelligence")]}

## Conversation Summary Memory

In [ ]:
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
summary_memory = ConversationSummaryMemory(llm=llm)
summary_memory.load_memory_variables({})

long_response = """Large Language Models (LLMs) have shown great promise as highly capable AI assistants that excel in
complex reasoning tasks requiring expert knowledge across a wide range of fields, including in specialized
domains such as programming and creative writing. They enable interaction with humans through intuitive
chat interfaces, which has led to rapid and widespread adoption among the general public.

The capabilities of LLMs are remarkable considering the seemingly straightforward nature of the training
methodology. Auto-regressive transformers are pretrained on an extensive corpus of self-supervised data,
followed by alignment with human preferences via techniques such as Reinforcement Learning with Human
Feedback (RLHF). Although the training methodology is simple, high computational requirements have
limited the development of LLMs to a few players.

There have been public releases of pretrained LLMs
(such as BLOOM (Scao et al., 2022), LLaMa-1 (Touvron et al., 2023), and Falcon (Penedo et al., 2023)) that
match the performance of closed pretrained competitors like GPT-3 (Brown et al., 2020) and Chinchilla
(Hoffmann et al., 2022), but none of these models are suitable substitutes for closed “product” LLMs, such
as ChatGPT, BARD, and Claude. These closed product LLMs are heavily fine-tuned to align with human
preferences, which greatly enhances their usability and safety. This step can require significant costs in
compute and human annotation, and is often not transparent or easily reproducible, limiting progress within
the community to advance AI alignment research.

In this work, we develop and release Llama 2, a family of pretrained and fine-tuned LLMs, Llama 2 and
Llama 2-Chat, at scales up to 70B parameters. On the series of helpfulness and safety benchmarks we tested,
Llama 2-Chat models generally perform better than existing open-source models. They also appear to
be on par with some of the closed-source models, at least on the human evaluations we performed (see
Figures 1 and 3). We have taken measures to increase the safety of these models, using safety-specific data
annotation and tuning, as well as conducting red-teaming and employing iterative evaluations. Additionally,
this paper contributes a thorough description of our fine-tuning methodology and approach to improving
LLM safety. We hope that this openness will enable the community to reproduce fine-tuned LLMs and
continue to improve the safety of those models, paving the way for more responsible development of LLMs.
We also share novel observations we made during the development of Llama 2 and Llama 2-Chat, such as
the emergence of tool usage and temporal organization of knowledge."""

In [ ]:
summary_memory.save_context({"input": "Tell me about Llama 2"}, {"output": long_response})
summary_memory.load_memory_variables({})

{'history': 'The AI explains that Large Language Models (LLMs) like Llama 2'}

In [ ]:
from langchain.memory import ConversationKGMemory
kgmemory = ConversationKGMemory(llm=llm, return_messages=True)
kgmemory.save_context({"input": "say hi to sam"}, {"output": "who is sam"})
kgmemory.save_context({"input": "sam is a friend"}, {"output": "okay"})
kgmemory.load_memory_variables({"input": "who is sam"})


{'history': [SystemMessage(content='On Sam: Sam is a friend.')]}

In [ ]:
kgmemory.get_knowledge_triplets("her favorite color is red")


[KnowledgeTriple(subject='Sam', predicate='favorite color is', object_='red')]